# Cloudfight Coding Contest AI 2022

In [89]:
# !pip install pandas
# !pip install matplotlib
# !pip install scikit-learn
# !pip install xgboost
# !pip install lightgbm


In [90]:
# Matrix and plots
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Models
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
DATA = r"abalone.data"
SEED = 2022


## Load data

In [91]:
df = pd.read_csv(DATA, header=None)
X = df.iloc[:, 0:-1]  # Get first k-1 cols
print(X.head())
y = df.iloc[:, -1]  # Get last col
print(y.head())


   0      1      2      3       4       5       6      7
0  M  0.455  0.365  0.095  0.5140  0.2245  0.1010  0.150
1  M  0.350  0.265  0.090  0.2255  0.0995  0.0485  0.070
2  F  0.530  0.420  0.135  0.6770  0.2565  0.1415  0.210
3  M  0.440  0.365  0.125  0.5160  0.2155  0.1140  0.155
4  I  0.330  0.255  0.080  0.2050  0.0895  0.0395  0.055
0    15
1     7
2     9
3    10
4     7
Name: 8, dtype: int64


## Preprocessing

### Missing values

In [92]:
# Replace numeric missing values by median
# X.loc[4177] = [None, None, None, None, None, None, None, None]
# numeric_attrs = X.select_dtypes(include=np.number).columns.tolist()
# imp = SimpleImputer(missing_values=np.nan, strategy='median')
# imp.fit(X.select_dtypes(include=np.number))
# X.iloc[:, numeric_attrs] = imp.transform(X.select_dtypes(include=np.number))

# Replace categorical missing values by mode


### Categorical attributes to numerical

In [93]:
enc = OneHotEncoder()
CAT_COLS = [0]
cat_cols_encoded = pd.DataFrame(enc.fit_transform(X[CAT_COLS]).toarray())
X = X.drop(columns=CAT_COLS)
X = pd.concat([X, cat_cols_encoded], axis=1)
X.columns = [str(i) for i in range(len(X.columns))]
print(X)


          0      1      2       3       4       5       6    7    8    9   10
0     0.455  0.365  0.095  0.5140  0.2245  0.1010  0.1500  0.0  0.0  1.0  0.0
1     0.350  0.265  0.090  0.2255  0.0995  0.0485  0.0700  0.0  0.0  1.0  0.0
2     0.530  0.420  0.135  0.6770  0.2565  0.1415  0.2100  1.0  0.0  0.0  0.0
3     0.440  0.365  0.125  0.5160  0.2155  0.1140  0.1550  0.0  0.0  1.0  0.0
4     0.330  0.255  0.080  0.2050  0.0895  0.0395  0.0550  0.0  1.0  0.0  0.0
...     ...    ...    ...     ...     ...     ...     ...  ...  ...  ...  ...
4173  0.590  0.440  0.135  0.9660  0.4390  0.2145  0.2605  0.0  0.0  1.0  0.0
4174  0.600  0.475  0.205  1.1760  0.5255  0.2875  0.3080  0.0  0.0  1.0  0.0
4175  0.625  0.485  0.150  1.0945  0.5310  0.2610  0.2960  1.0  0.0  0.0  0.0
4176  0.710  0.555  0.195  1.9485  0.9455  0.3765  0.4950  0.0  0.0  1.0  0.0
4177  0.545  0.425  0.140  0.7995  0.3360  0.1710  0.2340  0.0  0.0  0.0  1.0

[4178 rows x 11 columns]


In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=SEED)

ValueError: Found input variables with inconsistent numbers of samples: [4178, 4177]

## Model training

In [ ]:
# XGBoost
xgb_params = {"random_state": SEED}
xgb_model = XGBRegressor(**xgb_params)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
mean_squared_error(y_test, [round(i) for i in xgb_pred])


6.118201595358956